## Deploy custom container to AML endpoint

In [8]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
import os
import re
from datetime import datetime, timezone
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, ClientSecretCredential

In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
AZURE_TENANT_ID = os.environ.get("AZURE_TENANT_ID")
AZURE_SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
AZURE_CLIENT_ID = os.environ.get("AZURE_CLIENT_ID")
AZURE_CLIENT_SECRET = os.environ.get("AZURE_CLIENT_SECRET")
AZURE_ML_RESOURCE_GROUP = os.environ.get("AZURE_ML_RESOURCE_GROUP")
AZURE_ML_WORKSPACE = os.environ.get("AZURE_ML_WORKSPACE")
AZURE_CONTAINER_REGISTRY_NAME = os.environ.get("AZURE_CONTAINER_REGISTRY_NAME")
AZURE_CONTAINER_MODEL_NAMESPACE = os.environ.get("AZURE_CONTAINER_MODEL_NAMESPACE")

ENDPOINT_MODEL_NAME = os.environ.get("ENDPOINT_MODEL_NAME")

MODEL_NAME=os.environ.get("MODEL_NAME")
MODEL_VERSION=os.environ.get("MODEL_VERSION")

MODEL_IMAGE_NAME=os.environ.get("MODEL_IMAGE_NAME")
MODEL_IMAGE_VERSION=os.environ.get("MODEL_IMAGE_VERSION")

In [4]:
credential = DefaultAzureCredential()

credential = ClientSecretCredential(
    tenant_id=AZURE_TENANT_ID,
    client_id=AZURE_CLIENT_ID,
    client_secret=AZURE_CLIENT_SECRET,
)

ml_client = MLClient(
    credential, AZURE_SUBSCRIPTION_ID, AZURE_ML_RESOURCE_GROUP, AZURE_ML_WORKSPACE
)

In [5]:
# Creating a unique endpoint name with current datetime to avoid conflicts
online_endpoint_name = f"endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

print(online_endpoint_name)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=f"Endpoint for {MODEL_IMAGE_NAME}:{MODEL_IMAGE_VERSION}",
    auth_mode="key",
    tags={"model_name": MODEL_NAME, "model_version": MODEL_VERSION, "image_name": MODEL_IMAGE_NAME, "image_version": MODEL_IMAGE_VERSION},
)

endpoint-10300949462513


In [6]:
# Create the endpoint
results = ml_client.begin_create_or_update(endpoint).result()

In [10]:
# Save variables to the environment file
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
env_file = os.path.join(project_root, ".env")
iso_date_utc = datetime.now(timezone.utc).isoformat()
with open(env_file, 'a') as f:
    f.write("\n")
    f.write("# Script 03_deploy_endpoint.ipynb output variables\n")
    f.write(f"# Generated on {iso_date_utc}\n")
    f.write(f"AML_ENDPOINT_NAME={online_endpoint_name}\n")
    f.write(f"AML_ENDPOINT_SCORING_URI={results.scoring_uri}\n")
    f.write(f"AML_ENDPOINT_SCORING_URI={results.scoring_uri}\n")

In [11]:
f"blue-{MODEL_NAME}"

'blue-dev.ml_team.iris_classifier'

In [12]:
def clean_string(input_string):
    # Use regex to remove any character that is not a-z, A-Z, 0-9, or -
    cleaned_string = re.sub(r'[^a-zA-Z0-9-]', '', input_string)
    return cleaned_string

In [19]:
# create a blue deployment
model = ml_client.models.get(name=MODEL_NAME, version=MODEL_VERSION)
full_image_name = f"{AZURE_CONTAINER_REGISTRY_NAME}.azurecr.io/{AZURE_CONTAINER_MODEL_NAMESPACE}/{MODEL_IMAGE_NAME}:{MODEL_IMAGE_VERSION}"
deployment_name = clean_string(f'blue-{MODEL_NAME}')
env = Environment(
    image=full_image_name,
    inference_config={
        "liveness_route": {"port": 5001, "path": "/"},
        "readiness_route": {"port": 5001, "path": "/swagger.json"},
        "scoring_route": {"port": 5001, "path": "/score"},
    },
)
blue_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    instance_type="Standard_E16s_v3",
    instance_count=1,
)

In [20]:
ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint endpoint-10300949462513 exists


......................................................................

HttpResponseError: (BadArgument) Endpoint identity does not have pull permission on the registry. For System Assigned Identity case and workspace ACR, platform will wire up the permissions but if user is creating deployment with User Assigned Identity or if using non-workspace ACR, then User should grant pull permissions to identity on the registry. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#authorization-error
Code: BadArgument
Message: Endpoint identity does not have pull permission on the registry. For System Assigned Identity case and workspace ACR, platform will wire up the permissions but if user is creating deployment with User Assigned Identity or if using non-workspace ACR, then User should grant pull permissions to identity on the registry. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#authorization-error